# Ingesta de datos

Los datos serán los que ofrece la librería de *Yahoo Finance*, los cuales se obtienen mediante,
```
import yfinance as yf
```


# Valoración opciones aplicada

Tome en consideración el modelo continuio de Black and Scholes:

$$C(S,t)=S\cdot\Phi(d_1)-K\exp(-R\cdot t)\cdot\Phi(d_2)$$

donde $d_1=\frac{\log\frac{S}{K}+\left(R+\frac{\sigma^2}{2}\right)\cdot t}{\sqrt{\sigma^2\cdot t}}$ y $d_2=d_1-\sqrt{\sigma^2\cdot t}$.

Defina los parámetros asociados a valores del activo definido en la Tarea 1 y calcule el valor de una opción de compra.

### Cálculo tasa libre de riesgo

En esta parte se calcula la tasa libre de riesgo en base a un bono de tesorería de 10 años de USA.

In [14]:
import yfinance as yf

# Se define el ticker de un bono de tesorería de 10 años USA.
treasury_bond = '^TNX'  

treasury_bond = yf.Ticker(treasury_bond)
treasury_bond_data = treasury_bond.history(period="1y")

# Se obtiene el rendimiento más reciente.
rend = treasury_bond_data['Close'].iloc[-1]

risk_free_rate = rend / 100

print(f"Tasa libre de riesgo (Rendimiento bonos 10 años USA): {risk_free_rate:.2%}")

Tasa libre de riesgo (Rendimiento bonos 10 años USA): 4.92%


### Precio del ejercicio y Tiempo para expiración

Se obtienen datos relacionados para opciones de la acción del Banco de Chile que transa en NYSE.

In [15]:
import yfinance as yf
from datetime import datetime

# Replace 'BCH' with the appropriate ticker symbol for Banco de Chile.
ticker = 'BCH'

# Initialize a Yahoo Finance object for the stock.
stock = yf.Ticker(ticker)

# Fetch the option chain.
option_chain = stock.option_chain()

# Get the expiration dates of the options.
exp_dates = stock.options

time_to_expiration_years = [(datetime.strptime(exp_date, '%Y-%m-%d') - datetime.now()).days / 365 for exp_date in exp_dates]

strike_prices = option_chain.calls['strike']

print("Precios del ejercicio:")
print(strike_prices)

print("Tiempo hasta la expiración (en años):")
print(time_to_expiration_years)

Precios del ejercicio:
0    20.0
Name: strike, dtype: float64
Tiempo hasta la expiración (en años):
[0.0684931506849315, 0.2410958904109589, 0.4904109589041096]


### Cálculo volatilidad

En este caso se calcula la volatilidad de acuerdo a ventanas de tiempo que se van moviendo, y luego se calcula el promedio de las deviaciones estándar de cada ventana de tiempo. El parámetro, ```periodo_t``` sirve para definir cada cuántos días se calcula la desviación estándar, entonces si este es igual a 3, se calculará la desviación estándar cada 3 días.

In [16]:
import numpy as np
import scipy.stats as si

def volatilidad(ticker, periodo_t):
    df = yf.download(ticker, period = '1y')

    retornos_diarios = df['Adj Close'].pct_change()
    # Se calcula la desviación estándar para cada periodo_t. Y luego, se obtiene el promedio anualizado por la cantidad de días.
    sigma = retornos_diarios.rolling(periodo_t).std()
    # Volatilidad anualizada
    sigma = sigma.mean()*np.sqrt(periodo_t)
    return sigma

def black_scholes_call(S, K, T, r, sigma):
    # S: Precio actual del activo subyacente.
    # K: Precio del ejercicio de la opción.
    # T: Tiempo de que falta para la fecha de vencimiento.
    # r: Tasa libre de riesgo.
    # sigma: Volatilidad del activo subyacente.
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call = (S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    return call

ticker = 'BCH'
data = yf.download(ticker, period='1d')

# Precio actual de la acción del Banco de Chile.
S = data['Close'][-1]

# Precio del ejercicio.
K = strike_prices[len(strike_prices)-1]

# Tiempo que falta para la fecha de vencimiento (en años)
T = time_to_expiration_years[len(time_to_expiration_years)-1]

# Tasa libre de riesgo.
r = risk_free_rate

# Volatilidad del activo subyacente.
sigma = volatilidad(ticker, 3)

# Calculate call option price
call_option_price = black_scholes_call(S, K, T, r, sigma)

print(f'El precio para una opción call es: {call_option_price}')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
El precio para una opción call es: 0.13597314557145346


# Modelo alternativo

El siguiente modelo corresponde al modelo de valoración de opciones de Monte-Carlo.

In [17]:
import numpy as np

def monte_carlo_option_pricing_model(S, K, T, r, sigma, N, M, option_type='call', nivel_confianza=0.95):
    """
    S: Precio actual del activo.
    K: Precio del ejercicio.
    T: Tiempo para el vencimiento.
    r: Tasa libre de riesgo.
    sigma: Volatilidad del activo subyacente.
    N: Cantidad de pasos en el tiempo.
    M: Cantidad de simulaciones.
    option_type: Tipo de la opción ('call' o 'put').
    confidence_level: Nivel de confianza para el intervalo de confianza.
    """
    dt = T / N  # Paso tiempo
    I = np.zeros(M)  # Se inicializan los precios para cada simulación

    # Se simulan los M caminos del precio, para N pasos en el tiempo, cada uno.
    for i in range(M):
        S_t = S
        for j in range(N):
            S_t *= np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * np.random.standard_normal())
        print(S_t)
        I[i] = S_t

    # Se calculan los "pagos" en el vencimiento.
    if option_type == 'call':
        pago = np.maximum(I - K, 0)
    elif option_type == 'put':
        pago = np.maximum(K - I, 0)

    # Se descuentan los flujos para obtener el precio en t=0.
    C = np.exp(-r * T) * np.sum(pago) / M

    # Se calculan el precio promedio y el error estándar.
    precio_promedio = np.mean(pago)
    sd = np.std(pago, ddof=1) / np.sqrt(M)

    # Se calcula el intervalo de confianza.
    z = np.abs(np.percentile(np.random.normal(0, 1, M), (1 - nivel_confianza) * 100 / 2))
    lower_bound = precio_promedio - z * sd
    upper_bound = precio_promedio + z * sd

    return C, (lower_bound, upper_bound)


In [18]:
import numpy as np

def monte_carlo_option_pricing_model(S, K, T, r, sigma, N, M, option_type='call', nivel_confianza=0.95):
    """
    S: Precio actual del activo.
    K: Precio del ejercicio.
    T: Tiempo para el vencimiento.
    r: Tasa libre de riesgo.
    sigma: Volatilidad del activo subyacente.
    N: Cantidad de pasos en el tiempo.
    M: Cantidad de simulaciones.
    option_type: Tipo de la opción ('call' o 'put').
    confidence_level: Nivel de confianza para el intervalo de confianza.
    """
    dt = T / N  # Paso tiempo
    I = np.zeros(M)  # Se inicializan los precios para cada simulación

    # Se simulan los M caminos del precio, para N pasos en el tiempo, cada uno.
    for i in range(M):
        S_t = S
        for j in range(N):
            S_t *= np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * np.random.standard_normal())
        
        I[i] = S_t
    
    # Se calculan los "pagos" en el vencimiento.
    if option_type == 'call':
        pago = np.maximum(I - K, 0)
    elif option_type == 'put':
        pago = np.maximum(K - I, 0)

    # Se descuentan los flujos para obtener el precio en t=0.
    C = np.exp(-r * T) * np.sum(pago) / M

    # Se calculan el precio promedio y el error estándar.
    precio_promedio = np.mean(pago)
    sd = np.std(pago, ddof=1) / np.sqrt(M)

    # Se calcula el intervalo de confianza.
    z = np.abs(np.percentile(np.random.normal(0, 1, M), (1 - nivel_confianza) * 100 / 2))
    lower_bound = precio_promedio - z * sd
    upper_bound = precio_promedio + z * sd

    return C, (lower_bound, upper_bound)


In [19]:
import yfinance as yf
ticker = 'BCH'

data = yf.download(ticker,period = '1y')

# Obtenemos el precio actual de la acción.
S_0 = data['Adj Close'][-1]

# Tiempo para el vencimiento.
T = time_to_expiration_years[len(time_to_expiration_years)-1]
# Precio del ejercicio.
K = strike_prices[len(strike_prices)-1]
# Tasa libre de riesgo.
r = risk_free_rate
# Volatilidad de la acción.
sigma = volatilidad(ticker,3)
# Número de pasos del tiempo.
N = 12
# Cantidad de simulaciones.
M = 50
# Nivel de confianza.
confianza = 0.9

precio_opcion, ic = monte_carlo_option_pricing_model(S_0,K,T,r,sigma,N,M,'call',confianza)

print(f'Intervalo de confianza {confianza*100:.0f}%: [{ic[0]:.3f}, {ic[1]:.3f}]')
print(f'Promedio precio opción: {precio_opcion:.3f}')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
Intervalo de confianza 90%: [0.067, 0.138]
Promedio precio opción: 0.100
